<a href="https://www.kaggle.com/code/tanvikiran27/degeneracy-tables-with-api?scriptVersionId=248712379" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import itertools as it
import sympy as sp 
import pandas as pd
from sympy import symbols, Function, diff, FiniteSet, simplify, re, AlgebraicNumber
from sympy import Q, ask
from sympy import *
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import nC
from sympy.matrices import diag, eye

import os
import zipfile
import shutil

for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/degeneracy-tables-for-sphere-quotients/4D.csv
/kaggle/input/degeneracy-tables-for-sphere-quotients/5D.csv
/kaggle/input/degeneracy-tables-for-sphere-quotients/2D.csv
/kaggle/input/degeneracy-tables-for-sphere-quotients/3D.csv
/kaggle/input/degeneracy-tables-for-sphere-quotients/6D.csv
/kaggle/input/degeneracy-tables-for-sphere-quotients/7D.csv


In [2]:
# https://www.kaggle.com/code/nnjjpp/updating-a-dataset-with-a-notebook
from kaggle_secrets import UserSecretsClient
if os.path.exists('/root/.kaggle/'):
    pass
else:
    os.mkdir('/root/.kaggle/')
kaggle_API_key = UserSecretsClient().get_secret("key")

with open('/root/.kaggle/kaggle.json', 'w') as fid:
    # IMPORTANT: replace tanvikiran27 with your Kaggle username
    # Instructions for creating a Kaggle API Key: https://www.kaggle.com/docs/api#authentication
    fid.writelines(f'{{"username":"tanvikiran27","key":"{kaggle_API_key}"}}')

!chmod 600 /root/.kaggle/kaggle.json
print('Yay, it worked!')

Yay, it worked!


In [3]:
# https://www.kaggle.com/code/abdurrakibmollah/move-files-from-input-to-output-folder
dst_path = "/kaggle/working/"
for dirname, _, filenames in os.walk('/kaggle/input/degeneracy-tables-for-sphere-quotients'):
     for filename in filenames:
        src_path = os.path.join(dirname, filename)
        shutil.copy(src_path, dst_path)
        print('Copied '+filename)

Copied 4D.csv
Copied 5D.csv
Copied 2D.csv
Copied 3D.csv
Copied 6D.csv
Copied 7D.csv


# Link to dataset
[**Click here**]( https://kaggle.com/datasets/2bfac494d293fd0ef000308bde664b74a42f477733773b84662f2333be424ee1)

# Degeneracy tables

Remember to click Run All at the beginning of each session. To generate tables, make a new cell and call one of the functions below.

To generate a table for a lens space, use lens_table(k, ls, deg) \
    * k is the order of the group (usually prime) \
    * ls is the list of integers relatively prime to $k$ \
    * deg is how big you want the table to be \
    * **Example:** lens_table(5,[1,2,3],10) returns a table for L(5;1,2,3) containing entries for all $p,q \leq 10$

For a general quotient space, use degeneracy_table(generator, deg) \
    * generator is a matrix that generates the cyclic group you want to study \
    * deg is the same as above \
    * **Example:** degeneracy_table(M,10) returns a table for the quotient space under the group generated by the matrix $M$

## How to type Sympy matrices (spoiler: it's quite tedious)

eye(n) will give you the $n\times n$ identity matrix \
diag(x_1, x_2, ..., x_k) will give you a diagonal matrix with x_1,...,x_k along the main diagonal- and these entries can be matrices themselves!

For other matrices, use Matrix([[row 1],[row 2],...,[row k]]) \
    * Put rational entries inside of a Rational() object- e.g. Rational(1/2) instead of 1/2 (this makes sure the code performs calculations symbolically and doesn't use floating point representation) \
    * Use sqrt() for radicals \
    * I (capital i) is the imaginary number

See [Sympy's documentation](https://docs.sympy.org/latest/modules/matrices/matrices.html) for more information

In [4]:
# Example matrices
M1 = Matrix([[Rational(1/2),Rational(1/2),Rational(1/2),Rational(1/2)],
           [Rational(1/2),I/2,Rational(-1/2),-I/2],
           [Rational(1/2),Rational(-1/2),Rational(1/2),Rational(-1/2)],
           [Rational(1/2),-I/2,Rational(-1/2),I/2]])

M2 = Matrix([[Rational(1/2),sqrt(3)*I/2],[sqrt(3)*I/2,Rational(1/2)]])

# Change this to M2 to see the second matrix
M1

Matrix([
[1/2,  1/2,  1/2,  1/2],
[1/2,  I/2, -1/2, -I/2],
[1/2, -1/2,  1/2, -1/2],
[1/2, -I/2, -1/2,  I/2]])

In [5]:
# Returns a diagonal matrix that generates the cyclic group for a lens space
def make_lens_matrix(k:int, ls:[int]) -> sp.Matrix:
    M = Matrix()
    zeta = exp(2*pi*I/k)
    for l in ls:
        M=diag(M,zeta**l)
    return M

In [6]:
# Setting up symbols
z,w = symbols('z w')

# General Ikeda generating function for any FINITE group
def F_gen(group:sp.FiniteSet) -> sp.Add:
    F = 0
    for g in group:
        n = sqrt(len(g))
        frac = (1-z*w) / ((z*eye(n)-g).det() * (w*eye(n)-g.C).det())
        F += frac
    F = (1/len(group))*F
    return F

In [7]:
# Determines if a matrix is unitary
def is_unitary(g:sp.Matrix) -> bool:
    return g.inv() == g.H

# Helper function to check that the determinants aren't diverging
def get_abs(num:sp.Add) -> sp.Mul:
    return sqrt(re(((num)*conjugate(num)).expand()))

# Returns a set of matrices that form the cyclic group generated by g
def make_group(g:sp.Matrix) -> sp.FiniteSet:
    G = FiniteSet(eye(sqrt(len(g))))
    gamma = g 
    i=1
    unitary = is_unitary(g)
    while gamma != eye(sqrt(len(g))):
        G += FiniteSet(gamma)
        dist = get_abs(det(gamma)-1) # To make sure the group isn't infinite
        # print(dist)
        gamma *= g
        if get_abs(det(gamma)-1) > dist and not unitary:
            print(f'{get_abs(det(gamma)-1)}>{dist}')
            raise Exception("This is an infinite group. Please try a different generator.")
        i += 1
    return G 

In [8]:
# Coefficients from Taylor series expansion for a two-variable function- read more at
# https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
def taylor_coeffs(f:sp.Add, deg:int) -> list:
    table = []
    for i in range(deg+1):
        row = []
        for j in range(deg+1-i):
            d = diff(f,w,j)
            d = diff(d,z,i)
            d = lambdify([z,w],d)
            term = re(d(0,0)/(factorial(i)*factorial(j))).round()
            row.append(term)
        while len(row) < deg+1:
            row.append('-')
        table.append(row)
    return table

In [9]:
# Puts a '-' where there's no entry in the table
def add_dashes(table:list, deg:int) -> list:
    T = []
    for row in table:
        while len(row) < deg+1:
            row.append('-')
        T.append(row)
    return T

# Avoids repeating computations when making larger versions of tables that are already
# in the dataset
# Reads the existing table and adds onto it
def enlarge(f:sp.Add, deg:int, table:list) -> list:
    T = []
    for row in table:
        while row[-1]=='-':
            row = row[:-1]
        T.append(row)
    start = len(T)
    for i in range(start,deg+1):
        for j in range(i):
            d = diff(f,w,j)
            d = diff(d,z,i-j)
            d = lambdify([z,w],d)
            term = re(d(0,0)/(factorial(i-j)*factorial(j))).round()
            T[j].append(term)
        d = lambdify([z,w],diff(f,w,i))
        term = re(d(0,0)/(factorial(i))).round()
        T.append([term])
    T = add_dashes(T,deg)
    return T

In [10]:
# Gets a single degeneracy table (as a list of lists) from the dataset
def clean_col(df:pd.DataFrame, col:int) -> list:
    table = df[col].to_list()
    num_table = []
    while type(table[-1]) is float:
        table = table[:-1]
    for row in table:
        num_table.append(eval(row))
    return num_table

In [11]:
# Turns the order k and a list of l values into the name of a lens space (string)
# To avoid duplicates, the l values are modded by k and sorted in ascending order 
# (this doesn't affect the Ikeda generating function and thus doesn't affect the 
# degeneracy tables)
# However, a few duplicates found their way into the dataset from earlier versions (Sorry!)
def make_lens_str(k:int, ls:[int]) -> str:
    lens = "L("+str(k)+";"
    for i in range(len(ls)):
        ls[i] = ls[i] % k
    ls.sort()
    for l in ls:
        lens += str(l)+","
    lens = lens[:-1]
    lens += ")"
    return lens

In [12]:
# File lookup
def find_file(file:str) -> bool:
    for dirname, _, filenames in os.walk('/kaggle/working'):
        if file in filenames:
            return True
    return False

In [13]:
# Update dataset
# https://www.kaggle.com/code/nnjjpp/updating-a-dataset-with-a-notebook
def update_data():
    with open('/kaggle/working/dataset-metadata.json', 'w') as json_fid:
        json_fid.write('{\n  "title": "Degeneracy Tables for Sphere Quotients",\n  "id": "tanvikiran27/degeneracy-tables-for-sphere-quotients",\n  "licenses": [{"name": "CC0-1.0"}]}')
    !kaggle datasets version -m new

In [14]:
# In case you need to clear the dataset for some reason
# CAUTION: This will delete the .csvs from the output directory and can potentially
# delete everything in the dataset- use with caution...
# https://www.kaggle.com/code/focusedmonk/delete-output-files-in-kaggle
def clear_csvs():
    import pathlib
    files_to_delete = './*.csv' # this considers only ".csv" files. If you want to delete all files, use "./*"
    files_list = pathlib.Path(os.getcwd()).glob(files_to_delete)
    for file_path in files_list:
        os.remove(file_path)

In [15]:
# Helper function to extract k from L(k;l1,...,ln)
def get_order(lens:str) -> int:
    return int(lens[2:-1].split(';')[0])

# Tells where in the DataFrame to insert a new column
def get_spot(k:int,df:pd.DataFrame) -> int:
    if len(df.columns)!=0:
        c = 0
        while c<len(df.columns) and df.columns[c][0]!='M' and k>=int(get_order(df.columns[c])):
            c+=1
        return c
    return len(df.columns)

In [16]:
# Creates a degeneracy table for a lens space with entries for all 0 <= p,q <= deg
# If there's already an entry for the lens space, this function finds it in the dataset
# Otherwise it adds the new table to the dataset
def lens_table(k:int, ls:[int], deg:int) -> pd.DataFrame:
    N = len(ls)
    lens = make_lens_str(k,ls)
    # file = 'n=' + str(N) + '.csv'
    file = str(N) + 'D.csv'
    path = "/kaggle/working/" + file
    if find_file(file):
        try:
            data = pd.read_csv(path)
        except pd.errors.EmptyDataError:
            data = pd.DataFrame()
    else:
        data = pd.DataFrame()
    G = make_group(make_lens_matrix(k,ls))
    f = F_gen(G)
    if lens not in data:
        T = taylor_coeffs(f,deg)
        new = pd.DataFrame({lens:T})
        added = data.iloc[:,:get_spot(k,data)]
        right = data.iloc[:,get_spot(k,data):]
        added = added.merge(new, left_index = True, right_index = True, how = 'outer')
        added = added.merge(right,left_index = True, right_index = True, how = 'outer')
        added.to_csv(path, index = False)
        print("Created/updated " + file)
        update_data()
    else:
        T = clean_col(data,lens)
        if deg >= len(clean_col(data,lens)):
            T = enlarge(f,deg,T)
            data = data.drop(columns=[lens])
            added = data.iloc[:,:get_spot(k,data)]
            right = data.iloc[:,get_spot(k,data):]
            new = pd.DataFrame({lens:T})
            added = added.merge(new, left_index = True, right_index = True, how = 'outer')
            added = added.merge(right,left_index = True, right_index = True, how = 'outer')
            added.to_csv(path, index = False)
            print("Created/updated " + file)
            update_data()
    print(lens)
    return pd.DataFrame(T)

In [17]:
# Creates a table for any quotient space with entries for all 0 <= p,q <= deg
def degeneracy_table(generator:sp.Matrix, deg:int) -> pd.DataFrame:
    N = sqrt(len(generator))
    name = str(generator)
    file = str(N) + 'D.csv'
    path = "/kaggle/working/" + file
    if find_file(file):
        try:
            data = pd.read_csv(path)
        except pd.errors.EmptyDataError:
            data = pd.DataFrame()
    else:
        data = pd.DataFrame()
    G = make_group(generator)
    f = F_gen(G)
    if name not in data:
        T = taylor_coeffs(f,deg)
        new = pd.DataFrame({name:T})
        added = data.merge(new, left_index = True, right_index = True, how = 'outer')
        added.to_csv(path, index = False)
        print("Created/updated " + file)
        update_data()
    else:
        T = clean_col(data,name)
        if deg >= len(T):
            T = enlarge(f,deg,T)
            data = data.drop(columns=[name])
            new = pd.DataFrame({name:T})
            added = data.merge(new, left_index = True, right_index = True, how = 'outer')
            added.to_csv(path, index = False)
            print("Created/updated " + file)
            update_data()
    return pd.DataFrame(T)

In [18]:
# Adds the dimensions along a diagonal 
# Use this to see the relationship between box_b (H_p,q) and the real Laplacian (H_k)
def add_diag(table, diagonal:int) -> int:
    sum = 0
    for i in range(diagonal+1):
        sum += table[i][diagonal-i]
    return sum

# Prints out the sums of the diagonals in an aesthetically pleasing way
def print_diag_sums(table:list):
    for i in range(len(table)):
        print(f'p+q={i} -> Sum of diagonal = {add_diag(table,i)}')